# Import the libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

# Import the Data

2 Baskets of securities in the spreadsheet, PF1 and PF2

In sheet PF1
    a mix of stocks, commodities and stock indexes
    Amazon, American Airlines, SP500, Euronext100, Gold, Oil
    
In sheet PF2
    CenterPoint Energy(CNP), Ford, Walmart, General Electric, Tesla, Toyota
    
Change the sheet_name parameter below to be 'PF1' or 'PF2'

## Either from a spreadhsheet


In [ ]:
df = pd.read_excel(io='../Data/Portfolio.xlsx', sheet_name='PF2', index_col = 'Date')

# Display the first 5 and last 5 rows
pd.concat([df.head(), df.tail()])

## Download from Yahoo Finance

In [ ]:
import yfinance as yf

stocks = 'FB BTC-USD AMZN NFLX GOOGL TSLA F JPM GLD'.split()
df = yf.download(stocks, '2020-1-1')['Adj Close']

df = df[-253:]

df

# Plot some Charts

Nothing sophisticated, just enough to see how the various securities have performed

In [ ]:
num_securities = len(df.columns)
fig = plt.figure(figsize=(18,18))

for i in range(0, num_securities):
    ax = fig.add_subplot(4,3,i+1)
    ax.set_title(df.columns[i])
    ax.plot(df[df.columns[i]])

plt.show()

# Perform some calculations

- Daily returns
- Annual returns
- Daily covariance of daily returns
- Annual covariance of daily returns

**Note** these do not beed to be for a year, they do need to be for the period in question

In [ ]:
daily_ret = df.pct_change()

NUM_DAYS = daily_ret.count()

annual_ret = daily_ret.mean() * NUM_DAYS

cov_daily = daily_ret.cov()

cov_annual = cov_daily * NUM_DAYS

# Calculate the Efficient frontier

Obtained by simulating imaginary combinations of portfolios 
(50,000 portfolios of different combinations in this case).

In [ ]:
# set the number of combinations for imaginary portfolios
NUM_ASSETS = len(df.columns)
NUM_PORTFOLIOS = 50000


In [ ]:
# empty lists to store returns, volatility and weights of imiginary portfolios
port_returns = []
port_volatility = []
stock_weights = []


In [ ]:
# populate the empty lists with each portfolios returns,risk and weights

for portfolio in range(NUM_PORTFOLIOS):
    
    weights = np.random.random(NUM_ASSETS)
    weights /= np.sum(weights)
    
    returns = np.dot(weights, annual_ret)
    
    volatility = np.sqrt(np.dot(weights.T, np.dot(cov_annual, weights)))
    
    port_returns.append(returns)
    port_volatility.append(volatility)
    stock_weights.append(weights)
    
# Create a dictionary for Returns and Risk values of each portfolio
portfolio = {'Returns': port_returns,
             'Volatility': port_volatility}


In [ ]:
# extend original dictionary to accomodate each ticker and weight in the portfolio
assets = df.columns

for counter,asset in enumerate(assets):
    portfolio[asset+' Weight'] = [Weight[counter] for Weight in stock_weights]


In [ ]:
# make a nice dataframe of the extended dictionary
df_portfolio = pd.DataFrame(portfolio)

df_portfolio.head()

# Visualize the Efficient Frontier for this portfolio

In [ ]:
plt.style.use('seaborn')
df_portfolio.plot.scatter(x='Volatility', y='Returns', figsize=(10, 8), grid=True)

plt.xlabel('Volatility (Std. Deviation)')
plt.ylabel('Expected Returns')
plt.title('Efficient Frontier')
plt.show()

# Calculate the Sharpe Ratio

50,000 portfolios with different combinations of weights generated different expected returns and expected volatility. <BR>
Each point lying on the line represents an optimal combination of assets in a portfolio that maximises the expected return for the given level of risk <BR>
<BR><BR>
If all points on the Efficient Frontier curve are optimal combinations
- What is the optimal portfolio among the various optimal combinations? 
- What would even the criteria for choosing this *best of the best* combinations?
    
**Sharpe Ratio**<BR>
Use the `Sharpe Ratio` to measure the performance of an investments returns given its risk.
This ratio adjusts the returns of an investment which makes it possible to compare different investments on a scale that incorporates risk. <BR>
Without this scale of comparison, it would be virtually impossible to compare different investments with different combinations and their accompanying risks and returns.
    
    
The expected returns and returns of our simulated portfolios are already known. <BR>
Repeat the above process but include a calculation their risk-adjusted returns (with the help of the Sharpe Ratio), and make a colour bar of the plots using the Sharpe Ratio as the colour map

In [ ]:
# empty lists to store returns, volatility and weights of imiginary portfolios
port_returns = []
port_volatility = []
stock_weights = []
sharpe_ratio = []

#set random seed for reproduction's sake
np.random.seed(42)

for portfolio in range(NUM_PORTFOLIOS):
    
    weights = np.random.random(NUM_ASSETS)
    weights /= np.sum(weights)
    
    returns = np.dot(weights, annual_ret)
    
    volatility = np.sqrt(np.dot(weights.T, np.dot(cov_annual, weights)))
    
    sharpe = returns / volatility
    sharpe_ratio.append(sharpe)
    
    port_returns.append(returns)
    port_volatility.append(volatility)
    stock_weights.append(weights)
    
# Create a dictionary for Returns and Risk values of each portfolio
portfolio = {'Returns': port_returns,
             'Volatility': port_volatility,
             'Sharpe Ratio': sharpe_ratio}

# extend original dictionary to accomodate each ticker and weight in the portfolio
assets = df.columns

for counter,asset in enumerate(assets):
    portfolio[asset+' Weight'] = [Weight[counter] for Weight in stock_weights]
    
# make a nice dataframe of the extended dictionary
df_portfolio = pd.DataFrame(portfolio)


# Visualize the Efficient Frontier for this portfolio

In [ ]:
plt.style.use('seaborn-dark')
df_portfolio.plot.scatter(x='Volatility', y='Returns', c='Sharpe Ratio',
                          cmap='RdYlGn', edgecolors='black', figsize=(10, 8), grid=True)
plt.xlabel('Volatility (Std. Deviation)')
plt.ylabel('Expected Returns')
plt.title('Efficient Frontier')
plt.show()

# Locate the optimal Portfolio and least optimal portfilio


In [ ]:
# find min Volatility & max sharpe values in the dataframe 
is_min_vol = df_portfolio['Volatility'] ==  df_portfolio['Volatility'].min()
is_max_sharpe = df_portfolio['Sharpe Ratio'] == df_portfolio['Sharpe Ratio'].max()

# use the min, max values to locate and create the two special portfolios
max_sharpe_port = df_portfolio.loc[is_max_sharpe]
min_vol_port = df_portfolio.loc[is_min_vol]

# plot frontier, max sharpe & min Volatility values with a scatterplot
plt.style.use('seaborn-dark')
df_portfolio.plot.scatter(x='Volatility', y='Returns', c='Sharpe Ratio',
                          cmap='RdYlGn', edgecolors='black', figsize=(10, 8), grid=True)
plt.scatter(x=max_sharpe_port['Volatility'], y=max_sharpe_port['Returns'], c='red', marker='D', s=200)
plt.scatter(x=min_vol_port['Volatility'], y=min_vol_port['Returns'], c='blue', marker='D', s=200 )
plt.xlabel('Volatility (Std. Deviation)')
plt.ylabel('Expected Returns')
plt.title('Efficient Frontier')
plt.show()

To get specific details of these two special portfolios

In [ ]:
# print the details of the 2 special portfolios
print("Portfolio for Risk Averse investors")
print(min_vol_port.transpose())

print("\n\nPortfolio for Investors seeking max return")
print(max_sharpe_port.transpose())

**Risk-averse investors** <BR>
would construct the minimum volatility portfolio 

**Investors seeking maximum risk-adjusted return** <BR>
would construct the max Sharpe Ratio portfolio 

In [ ]:
type(min_vol_port)

In [ ]:
min_vol_port.index

In [ ]:
min_vol_port.transpose().index.values